**Nisarg Rade Assignment**

Connect google drive to download / load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Run this cell if data is not downloaded
!mkdir -p /content/drive/MyDrive/parallel_dots/
!wget -P /content/drive/MyDrive/parallel_dots/ http://www.cs.cmu.edu/~ehsiao/3drecognition/CMU10_3D.zip
!pip install -r /content/drive/MyDrive/parallel_dots/requirements.txt
TARGET_FOLDER = "/content/drive/MyDrive/parallel_dots/"

# Clone the repository into the target folder
!git clone https://github.com/facebookresearch/segment-anything-2.git $TARGET_FOLDER

In [ ]:
# comment if not using colab
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything-2.git'
else:
  !git clone https://github.com/facebookresearch/segment-anything.git
  !cd segment-anything
  !pip install -e .



PyTorch version: 2.4.0+cu121
Torchvision version: 0.19.0+cu121
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything-2.git to /tmp/pip-req-build-94wq6u4d
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2.git /tmp/pip-req-build-94wq6u4d
  Resolved https://github.com/facebookresearch/segment-anything-2.git to commit 7e1596c0b6462eb1d1ba7e1492430fed95023598
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.5 MB/s eta 0:00:00
  Created w

 Import necessary libraries

In [ ]:
import torch
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2_video_predictor
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import os, glob, shutil
import matplotlib.patches as patches
import re
import time
import cv2

import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

/usr/local/lib/python3.10/dist-packages/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()


using device: cuda


load the sam2 model for prediction
please provide correct path for the model if stored at different place

In [ ]:
checkpoint = "/content/drive/MyDrive/parallel_dots/segment-anything-2/checkpoints/sam2_hiera_tiny.pt"
model_cfg = "sam2_hiera_t.yaml"
tempfolder = "./tempdir"
# Initialize SAM2 model and predictors
sam2 = build_sam2(model_cfg, checkpoint, device='cuda', apply_postprocessing=False)
predictor_prompt = SAM2ImagePredictor(build_sam2(model_cfg, checkpoint))
mask_generator = SAM2AutomaticMaskGenerator(sam2)
predictor_vid = build_sam2_video_predictor(model_cfg, checkpoint, device='cuda')

Create required functions for SAM2 detection

In [ ]:
def create_if_not_exists(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

def cleardir(tempfolder):
    filepaths = glob.glob(tempfolder + "/*")
    for filepath in filepaths:
        os.unlink(filepath)
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.get_cmap("tab10")
        cmap_idx = 0 if obj_id is None else obj_id
        color = np.array([*cmap(cmap_idx)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))
def get_mask_bounding_box(mask):
    # Find indices where the mask is non-zero
    if np.sum(mask) == 0:
        print("Empty mask detected, cannot compute bounding box.")
        return None

    # Find indices where the mask is non-zero
    mask_indices = np.argwhere(mask)

    if mask_indices.size == 0:
        print("No non-zero indices found in the mask.")
        return None
    mask_indices = np.argwhere(mask)
    ymin, xmin = mask_indices.min(axis=0)
    ymax, xmax = mask_indices.max(axis=0)
    return xmin, ymin, xmax, ymax

## **Implemented the function to get the point from first_image/mask**

In [ ]:
def process_img_png_mask(image_path, mask_path, visualize=False):
    # Add code to load images and masks, process them, and visualize if needed
    # Sample implementation (adjust based on assignment requirements)
    img = Image.open(image_path)
    mask = Image.open(mask_path)
    mask_np = np.array(mask)

    if visualize:
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(img)
        ax[1].imshow(mask_np, cmap='gray')
        plt.show()

    # Extract bounding box (xmin, xmax, ymin, ymax)
    ymin, ymax = np.min(np.where(mask_np)[0]), np.max(np.where(mask_np)[0])
    xmin, xmax = np.min(np.where(mask_np)[1]), np.max(np.where(mask_np)[1])
    print([xmin, xmax, ymin, ymax])
    return [xmin, xmax, ymin, ymax]

Created a function to detect the mask and return the bounding box

In [ ]:
def track_item_boxes(imgpath1, imgpath2, img1boxclasslist, visualize=True):
    create_if_not_exists(tempfolder)
    cleardir(tempfolder)
    shutil.copy(imgpath1, tempfolder + "/00000.jpg")
    shutil.copy(imgpath2, tempfolder + "/00001.jpg")

    inference_state = predictor_vid.init_state(video_path="./tempdir")
    predictor_vid.reset_state(inference_state)
    ann_frame_idx = 0

    for img1boxclass in img1boxclasslist:
        ([xmin, xmax, ymin, ymax], objectnumint) = img1boxclass
        box = np.array([xmin, ymin, xmax, ymax], dtype=np.float32)
        _, out_obj_ids, out_mask_logits = predictor_vid.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=ann_frame_idx,
            obj_id=objectnumint,
            box=box,
        )

    video_segments = {}
    bounding_boxes = {}
    for out_frame_idx, out_obj_ids, out_mask_logits in predictor_vid.propagate_in_video(inference_state):
        video_segments[out_frame_idx] = {
            out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
            for i, out_obj_id in enumerate(out_obj_ids)
        }

        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            if out_mask.sum() > 0:  # Check if the mask has any positive values
                bbox = get_mask_bounding_box(out_mask[0])
                if bbox is not None:
                    bounding_boxes[out_obj_id] = bbox

    if visualize:
        fig, ax = plt.subplots()
        plt.title("Original image object ::")
        ax.imshow(Image.open(tempfolder + "/00000.jpg"))
        rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='g', facecolor='none')
        ax.add_patch(rect)
        plt.show()

        out_frame_idx = 1
        plt.figure(figsize=(6, 4))
        plt.title("Detected object in test image ::")
        plt.imshow(Image.open(tempfolder + "/00001.jpg"))
        for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            if out_mask.sum() > 0:  # Check if the mask has any positive values
                show_mask(out_mask, plt.gca(), obj_id=out_obj_id)

    print(bounding_boxes)
    return video_segments, bounding_boxes


### **Created functions to get object detection performance for each product**

In [ ]:
# Function to evaluate detection performance
def evaluate_detection_performance(pred_boxes, gt_boxes):
    ious = []
    for pred_box, gt_box in zip(pred_boxes, gt_boxes):
        if pred_box is None:  # Handle missing detections
            ious.append(0)  # IoU is 0 when the detection fails
        else:
            iou = calculate_iou(pred_box, gt_box)
            ious.append(iou)
    mean_iou = np.mean(ious)
    precision = sum([iou >= 0.5 for iou in ious]) / len(ious) if ious else 0
    recall = sum([iou >= 0.5 for iou in ious]) / len(gt_boxes) if gt_boxes else 0
    return {
        "mean_iou": mean_iou,
        "precision": precision,
        "recall": recall
    }

# IoU Calculation
def calculate_iou(boxA, boxB):
    if boxA is None or boxB is None:
        return 0  # Return 0 IoU when any box is None
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Example use case with images (replace with actual paths and details as required)
def detect_objects(reference_img, reference_mask, target_img):
  # firstimgpath = '/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000001.jpg'
  # firstimgmaskpath = '/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000001_1_gt.png'
  firstimgpath=reference_img
  firstimgmaskpath=reference_mask
  [xmin, xmax, ymin, ymax] = process_img_png_mask(firstimgpath, firstimgmaskpath, visualize=False)

  # secondimgpath = '/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000014.jpg'
  secondimgpath=target_img
  secondimg = Image.open(secondimgpath)


  op,bounding_box = track_item_boxes(firstimgpath, secondimgpath, [([xmin, xmax, ymin, ymax], 1)], visualize=False)
  # print(op)
  output_masks = op[1]  # Example code assumes masks are stored this way
  # print(output_masks)
  if output_masks is None or np.sum(output_masks) == 0:
        return None
  relevant_mask = output_masks[1]

  # print()
  return bounding_box[1]
# print(relevant_mask)
def calculate_box_distance(box1, box2):
    # Calculate the distance between the centers of the two bounding boxes
    x1, y1, x2, y2 = box1
    x1_gt, y1_gt, x2_gt, y2_gt = box2
    center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
    center2 = ((x1_gt + x2_gt) / 2, (y1_gt + y2_gt) / 2)
    return np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)

Main code to drive through the first image/mask of every product and perform detection as save the results in a txt file

In [ ]:

# Directory paths
data_path = '/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D'  # Replace with the correct path
output_path = "/content/output"  # Directory to save results

# Define the initial reference images and masks
reference_pairs = {
    "can_chowder": ("can_chowder_000001.jpg", "can_chowder_000001_1_gt.png"),
    "can_soymilk": ("can_soymilk_000001.jpg", "can_soymilk_000001_1_gt.png"),
    "can_tomatosoup": ("can_tomatosoup_000001.jpg", "can_tomatosoup_000001_1_gt.png"),
    "carton_oj": ("carton_oj_000001.jpg", "carton_oj_000001_1_gt.png"),
    "carton_soymilk": ("carton_soymilk_000001.jpg", "carton_soymilk_000001_1_gt.png"),
    "diet_coke": ("diet_coke_000001.jpg", "diet_coke_000001_1_gt.png"),
    "hc_potroastsoup": ("hc_potroastsoup_000001.jpg", "hc_potroastsoup_000001_1_gt.png"),
    "juicebox": ("juicebox_000001.jpg", "juicebox_000001_1_gt.png"),
    "rice_tuscan": ("rice_tuscan_000001.jpg", "rice_tuscan_000001_1_gt.png"),
    "ricepilaf": ("ricepilaf_000001.jpg", "ricepilaf_000001_1_gt.png")
}

results = {}
for object_name, (ref_img_name, ref_mask_name) in reference_pairs.items():
    # Load reference image and mask
    ref_img_path = os.path.join(data_path, ref_img_name)
    ref_mask_path = os.path.join(data_path, ref_mask_name)
    reference_img = cv2.imread(ref_img_path)
    reference_mask = cv2.imread(ref_mask_path, cv2.IMREAD_GRAYSCALE)

    # Get all images of the current object type
    object_images = [img for img in os.listdir(data_path) if img.startswith(object_name) and img.endswith('.jpg')]

    pred_boxes = []
    gt_boxes = []

    for img_name in object_images:
        img_path = os.path.join(data_path, img_name)
        mask_path = img_path.replace(".jpg", "_1_gt.png")
        print(img_path,mask_path)
        # Load target image and ground truth mask
        target_img = img_path
        gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Detect object using the first reference pair
        pred_box = detect_objects(ref_img_path, ref_mask_path, target_img)
        if pred_box is None :
            pred_boxes.append(None)
            gt_boxes.append(None)
            continue
        if str(pred_box) == "(373, 154, 463, 290)":
            print("appending none")
            pred_boxes.append(None)
            gt_boxes.append(None)
            continue

        # Initialize variables to find the closest mask
        best_gt_box = None
        min_distance = float('inf')

        # Iterate through possible mask files
        for i in range(1, 4):
            mask_path = img_path.replace(".jpg", f"_{i}_gt.png")
            if os.path.exists(mask_path):
                gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                gt_box = get_mask_bounding_box(gt_mask)

                if gt_box is not None:
                    distance = calculate_box_distance(pred_box, gt_box)
                    if distance < min_distance:
                        min_distance = distance
                        best_gt_box = gt_box

        pred_boxes.append(pred_box)
        gt_boxes.append(best_gt_box)


    # Calculate evaluation metrics
    performance = evaluate_detection_performance(pred_boxes, gt_boxes)
    results[object_name] = performance
    print(f"Performance for {object_name}: {performance}")

# Save results
with open(os.path.join(output_path, "detection_performance.txt"), "w") as f:
    for object_name, performance in results.items():
        f.write(f"{object_name}: {performance}\n")

print("Object detection performance evaluation completed.")

/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000022.jpg /content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000022_1_gt.png


propagate in video: 100%|██████████| 2/2 [00:00<00:00,  2.30it/s]


{1: (150, 228, 293, 364)}
/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000033.jpg /content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000033_1_gt.png


propagate in video: 100%|██████████| 2/2 [00:00<00:00,  2.29it/s]


{1: (373, 154, 463, 290)}
appending none
/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000006.jpg /content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/can_chowder_000006_1_gt.png


...

{1: (254, 202, 433, 390)}
/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/ricepilaf_000037.jpg /content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/ricepilaf_000037_1_gt.png


propagate in video: 100%|██████████| 2/2 [00:00<00:00,  2.13it/s]


{1: (418, 108, 503, 241)}
/content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/ricepilaf_000032.jpg /content/drive/MyDrive/parallel_dots/CMU10_3D/CMU10_3D/data_2D/ricepilaf_000032_1_gt.png


propagate in video: 100%|██████████| 2/2 [00:00<00:00,  2.12it/s]

{1: (418, 108, 503, 241)}
Performance for ricepilaf: {'mean_iou': 0.3519034375335188, 'precision': 0.34, 'recall': 0.34}
Object detection performance evaluation completed.


Display the performance result

In [2]:

file_path = '/content/outputdetection_performance.txt'

# Open and read the file
with open(file_path, 'r') as file:
    content = file.read()

# Display the content
print(content)


diet_coke: {'mean_iou': 0.7600907878025357, 'precision': 0.84, 'recall': 0.84}
hc_potroastsoup: {'mean_iou': 0.7568471381463258, 'precision': 0.82, 'recall': 0.82}
rice_tuscan: {'mean_iou': 0.5685131976807216, 'precision': 0.58, 'recall': 0.58}
carton_soymilk: {'mean_iou': 0.5102140608757126, 'precision': 0.54, 'recall': 0.54}
can_chowder: {'mean_iou': 0.3875101071600811, 'precision': 0.44, 'recall': 0.44}
carton_oj: {'mean_iou': 0.36959294859602726, 'precision': 0.4, 'recall': 0.4}
ricepilaf: {'mean_iou': 0.3519034375335188, 'precision': 0.34, 'recall': 0.34}
can_soymilk: {'mean_iou': 0.34217088830525144, 'precision': 0.34, 'recall': 0.34}
can_tomatosoup: {'mean_iou': 0.29593599595804176, 'precision': 0.28, 'recall': 0.28}
juicebox: {'mean_iou': 0.24683781655947798, 'precision': 0.2, 'recall': 0.2}

